In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from collections import defaultdict
from collections.abc import Iterable
import inspect
import matplotlib.pyplot as plt
import numpy as np
from operator import gt, lt, add, sub
import os
import pandas as pd
from tabulate import tabulate
from sklearn.metrics import (accuracy_score, dcg_score, roc_auc_score, 
                             precision_score, recall_score)
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from htools import hdir, LoggerMixin, eprint
from ml_htools.torch_utils import ModelMixin, variable_lr_optimizer, DEVICE, stats
from spellotape.utils import stop_instance

In [3]:
# Reproducible testing.
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True

# To Do:

- Maybe get different logger and make new folder and/or file for each training run?
- Finish + test csvlogger (decide whether to comebine with statshandler)
- Build + add + test LRScheduler
- Test regression

In [4]:
class Data(Dataset):
    
    def __init__(self, n=64, dim=2):
        self.x = torch.rand(n, dim).float()
        self.y = torch.clamp(
            (self.x[:, 0]*.75 + self.x[:, 1]*.25).round(), 0, 1
        ).abs().unsqueeze(-1)
        
    def __getitem__(self, i):
        return self.x[i], self.y[i]
    
    def __len__(self):
        return len(self.x)

In [44]:
class Model(nn.Module, LoggerMixin):
    
    def __init__(self, dim, criterion, path=os.path.join('..', 'data'),
                 callbacks=None, metrics=None):
        super().__init__()
        self.fc = nn.Linear(dim, 1)
        self.criterion = criterion    
        # Dictionary makes it easier to adjust callbacks after creating model.
        callbacks = [ModelHandler(), StatsHandler(), MetricPrinter()] \
                    + (callbacks or [])
        self.callbacks = {type(cb).__name__: cb for cb in callbacks}
        self.metrics = [batch_size] + (metrics or [])
        self.logger = self.get_logger(os.path.join(path, 'train.log'), 
                                      fmt='\n%(asctime)s\n %(message)s')
        
    def forward(self, x):
        return self.fc(x)
    
    def fit(self, epochs, loaders, lrs, optim=None, callbacks=None, 
            metrics=None, classify=True, logit=True, thresh=.5,
            device=DEVICE):
        # Initialize stats, data loaders, optimizer, and callbacks.
        stats = defaultdict(list)
        train_dl, val_dl = loaders
        optim = optim or variable_lr_optimizer(self, lrs=lrs)
        _ = self.stop_training('on_train_begin', callbacks, metrics)
            
        # Train.
        for epoch in range(1, epochs+1):
            _ = self.stop_training('on_epoch_begin', epoch, stats)
            for i, (xb, yb) in enumerate(train_dl, 1):
                xb, yb = xb.to(device), yb.to(device)
                optim.zero_grad()
                _ = self.stop_training('on_batch_begin')
                
                # Forward and backward passes.
                y_score = self(xb)
                loss = self.criterion(y_score, yb)
                loss.backward()
                optim.step()
                
                # Separate because callbacks are only applied during training.
                self._update_stats(stats, loss, yb, y_score.detach(),
                                   classify, logit, thresh)
                if self.stop_training('on_batch_end', stats): break
            
            # If on_batch_end callback halts training, else block is skipped.  
            else: 
                val_stats = self.validate(val_dl, classify, logit, thresh)
                if self.stop_training('on_epoch_end', epoch, stats, val_stats):
                    break
                continue
            break      

        self.stop_training('on_train_end', stats, val_stats)
            
    def validate(self, val_dl, classify, logit, thresh):
        val_stats = defaultdict(list)
        self.eval()
        with torch.no_grad():
            for xb, yb in val_dl:
                y_score = self(xb)
                loss = self.criterion(y_score, yb)
                self._update_stats(val_stats, loss, yb, y_score, classify,
                                   logit, thresh)
        return val_stats
    
    def _update_stats(self, stats, loss, yb, y_score, classify, logit, thresh):
        """Update stats in place.
        
        Parameters
        ----------
        stats: defaultdict[str, list]
        loss: torch.Tensor
            Tensor containing single value (mini-batch loss).
        yb: torch.Tensor
            Mini-batch of labels.
        y_pred: torch.Tensor
            Mini-batch of predictions.
            
        Returns
        -------
        None
        """
        if classify:
            if logit: y_score = torch.sigmoid(y_score)
            y_pred = (y_score > thresh).float()
            
        stats['loss'].append(loss.detach().cpu().numpy().item())
        for m in self.metrics:
            yhat = y_pred if hasarg(m, 'y_pred') else y_score
            stats[m.__name__.replace('_score', '')].append(m(yb, yhat))
    
    def stop_training(self, attr, *args, **kwargs):
        self._stop_training = False
        # Pass model object as first argument to callbacks.
        for cb in self.callbacks.values():
            getattr(cb, attr)(self, *args, **kwargs)
        return self._stop_training
    
    def dims(self):
        """Get shape of each layer's weights."""
        return [tuple(p.shape) for p in self.parameters()]

    def trainable(self):
        """Check which layers are trainable."""
        return [(tuple(p.shape), p.requires_grad) for p in self.parameters()]

    def weight_stats(self):
        """Check mean and standard deviation of each layer's weights."""
        return [stats(p.data, 3) for p in self.parameters()]

    def plot_weights(self):
        """Plot histograms of each layer's weights."""
        n_layers = len(self.dims())
        fig, ax = plt.subplots(n_layers, figsize=(8, n_layers * 1.25))
        if not isinstance(ax, Iterable): ax = [ax]
        for i, p in enumerate(self.parameters()):
            ax[i].hist(p.data.flatten())
            ax[i].set_title(f'Shape: {tuple(p.shape)} Stats: {stats(p.data)}')
        plt.tight_layout()
        plt.show()

# Callbacks

In [25]:
class TorchCallback:
    
    def on_train_begin(self, model, callbacks, metrics):
        pass
    
    def on_train_end(self, model, stats, val_stats):
        pass
    
    def on_epoch_begin(self, model, epoch, stats):
        pass

    def on_epoch_end(self, model, epoch, stats, val_stats):
        pass
    
    def on_batch_begin(self, model):
        pass
    
    def on_batch_end(self, model, stats):
        pass

In [26]:
class EarlyStopper(TorchCallback):
    
    def __init__(self, goal, stat='loss', min_improvement=0.0, patience=3):
        """
        Parameters
        ----------
        goal: str
            Indicates what we want to do to the metric in question.
            Either 'min' or 'max'. E.g. metric 'loss' should have goal 'min'
            while metric 'precision' should have goal 'max'.
        stat: str
            Quantity to monitor. This will always be computed on the 
            validation set.
        min_improvement: float
            Amount of change needed to qualify as improvement. For example,
            min_improvement of 0.0 means any improvement is sufficient. With
            a min_improvent of 0.2, we will stop training even if the
            quantity improves by, for example, 0.1.
        patience: int
            Number of acceptable epochs without improvement. E.g. patience=0 
            means the metric must improve every epoch for training to continue.            
        """
        # Will use op like: self.op(new_val, current_best)
        if goal == 'min':
            self.init_stat = self.best_stat = float('inf')
            self.op = lt
            self.op_best = sub
        elif goal == 'max':
            self.init_stat = self.best_stat = float('-inf')
            self.op = gt
            self.op_best = add
        else:
            raise ValueError('Goal must be "min" or "max".')
            
        self.stat = stat
        self.min_improvement = min_improvement
        self.patience = patience
        self.since_improvement = 0
        
    def on_train_begin(self, model, callbacks, metrics):
        """Resets tracked variables at start of training."""
        self.best_stat = self.init_stat
        self.since_improvement = 0
    
    def on_epoch_end(self, model, epoch, stats, val_stats):
        new_val = val_stats.get(self.stat, None)
        if new_val is None:
            model.logger.info(f'EarlyStopper could not find {self.stat}.'
                              f'Callback behavior may not be enforced.')
            
        if self.op(new_val, self.op_best(self.best_stat, self.min_improvement)):
            self.best_stat = new_val
            self.since_improvement = 0
        else:
            self.since_improvement += 1
            if self.since_improvement > self.patience:
                model.logger.info(
                    f'EarlyStopper halting training: validation {self.stat} '
                    f'has not improved enough in {self.since_improvement} epochs.'
                )
                model._stop_training = True

In [27]:
class PerformanceThreshold(TorchCallback):
    
    def __init__(self, metric, goal, threshold, split='val'):
        assert split in ('train', 'val'), 'Split must be "train" or "val".'
        assert goal in ('min', 'max'), 'Goal must be "min" or "max"'
        
        self.metric = metric
        self.threshold = threshold
        self.split = split
        self.op = gt if goal == 'min' else lt
        
    def on_epoch_end(self, model, epoch, stats, val_stats):
        data = val_stats if self.split == 'val' else stats
        new_val = data.get(self.metric, None)
        if new_val is None:
            model.logger.info(f'{self.metric} not found in metrics.'
                              'PerformanceThreshold may not be enforced.')
            return
        
        if self.op(new_val, self.threshold):
            model.logger.info(
                f'PerformanceThreshold halting training: {self.metric} '
                f'of {new_val:.4f} did not meet threshold.'
            )
            model._stop_training = True

In [28]:
class MetricPrinter(TorchCallback):
    
    def on_epoch_end(self, model, epoch, stats, val_stats):
        data = [[k, v, val_stats[k]] for k, v in stats.items()]
        table = tabulate(data, headers=['Metric', 'Train', 'Validation'], 
                         tablefmt='github', floatfmt='.4f')
        model.logger.info(f'Epoch {epoch}\n\n{table}\n\n{"="*9}')

In [60]:
class ModelHandler(TorchCallback):
    """Handles basic model tasks like putting the model on the GPU
    and switching between train and eval modes.
    """
    
    def on_train_begin(self, model, callbacks, metrics):
        model.to(DEVICE)
        if callbacks: model.callbacks.update(
            {type(cb).__name__: cb for cb in callbacks}
        )
        if metrics: model.metrics.extend(metrics)
        
    def on_epoch_begin(self, model, epoch, stats):
        model.train()
        
    def on_train_end(self, model, stats, val_stats):
        model.logger.info('Training complete. Model in eval mode.')
        model.eval()

In [46]:
class StatsHandler(TorchCallback):
        
    def on_epoch_begin(self, model, epoch, stats):
        stats.clear()
        
    def on_epoch_end(self, model, epoch, stats, val_stats):
        for group in (stats, val_stats):
            for k, v in group.items():
                if k == 'batch_size': continue
                group[k] = np.average(v, weights=group['batch_size'])
            group.pop('batch_size')

In [47]:
class CSVLogger(TorchCallback):
    """Separate from StatsHandler in case we don't want to log outputs."""
    
    def __init__(self, mode='epoch', file_fmt='{}_stats.csv'):
        assert mode in ('epoch', 'batch'), \
            'Mode must be "epoch" or "batch".'
        self.mode = mode
        self.history = defaultdict(list)
        self.fname = file_fmt.format(mode)
        
    def on_train_begin(self, model, callbacks, metrics):
        pass
        
    def on_batch_end(self, model, stats):
        if self.mode != 'batch':
            pass
        pass
        
    def on_epoch_end(self, model, epoch, stats, val_stats):
        if self.mode != 'epoch':
            pass
        
    def write_csv(self):
        pass

In [48]:
class EC2Closer(TorchCallback):
    
    def on_train_end(self, model, stats, val_stats):
        stop_instance()

In [49]:
type(PerformanceThreshold('loss', 'min', 0.5)).__name__

'PerformanceThreshold'

# Metrics

Keep sklearn pattern with y_true as first argument.

For classification problems, round probabilities once instead of in every metric.

In [50]:
def hasarg(func, arg):
    return arg in inspect.signature(func).parameters

In [51]:
def percent_positive(y_true, y_pred):
    return (y_pred == 1).float().mean()

In [52]:
def mean_soft_prediction(y_true, y_score):
    return y_score.mean() 

In [53]:
def batch_size(y_true, y_pred):
    return y_true.shape[0]

In [54]:
[hasarg(roc_auc_score, val) for val in ('y_score', 'y_pred')]

[True, False]

In [55]:
[hasarg(precision_score, val) for val in ('y_score', 'y_pred')]

[False, True]

# Training

In [56]:
DIM = 2
metrics = [accuracy_score, 
           precision_score, 
           recall_score, 
           percent_positive,
           mean_soft_prediction
          ]
callbacks = [EarlyStopper('max', 'accuracy', patience=3),
             PerformanceThreshold('recall', 'max', 0.25)]

In [57]:
train = Data(n=34, dim=DIM)
val = Data(n=30, dim=DIM)

dl_train = DataLoader(train, batch_size=8, shuffle=True)
dl_val = DataLoader(val, batch_size=8, shuffle=False)

In [58]:
net = Model(DIM, F.binary_cross_entropy_with_logits, callbacks=callbacks,
            metrics=metrics)
net

Model(
  (fc): Linear(in_features=2, out_features=1, bias=True)
)

In [59]:
net.fit(10, [dl_train, dl_val], [.3])

None

2020-02-10 17:20:19,707
 Epoch 1

| Metric               |   Train |   Validation |
|----------------------|---------|--------------|
| loss                 |  0.6467 |       0.5533 |
| accuracy             |  0.6471 |       0.8333 |
| precision            |  0.4235 |       0.8333 |
| recall               |  0.7059 |       0.8000 |
| percent_positive     |  0.5000 |       0.3667 |
| mean_soft_prediction |  0.5016 |       0.4430 |


2020-02-10 17:20:19,735
 Epoch 2

| Metric               |   Train |   Validation |
|----------------------|---------|--------------|
| loss                 |  0.5265 |       0.4700 |
| accuracy             |  0.8235 |       0.9000 |
| precision            |  0.7686 |       0.8111 |
| recall               |  0.7569 |       1.0000 |
| percent_positive     |  0.4412 |       0.5000 |
| mean_soft_prediction |  0.4921 |       0.5264 |


2020-02-10 17:20:19,763
 Epoch 3

| Metric               |   Train |   Validation |
|----------------------|---------|----